In [1]:
import pandas as pd
import os,re,sys,subprocess
import pysrt
import glob
import csv

In [2]:
links = pd.read_csv('video_links_1.csv',encoding='utf-8')
links = links.dropna(axis=1, how='all')
category = links.category.unique()

In [9]:
EmployeeList =list(links.source_product_name)
[str(x) for x in EmployeeList]

['Luxembourg Gardens',
 'Buckingham Palace',
 'Eiffel Tower',
 'Churchill War Rooms',
 'Thames Rockets',
 'Fat Tire Tours Berlin',
 "Musee d'Orsay",
 'National Gallery']

In [10]:
links

,product_id,source_product_name,city,category,URLS
0,2600006,Luxembourg Gardens,Paris,nature_parks,https://www.youtube.com/watch?v=fQAAMSedWS0
1,2600023,Buckingham Palace,London,nature_parks,https://www.youtube.com/watch?v=T_8kE_JFUhY
2,2500009,Eiffel Tower,Paris,landmarks,https://www.youtube.com/watch?v=pvZRgAZ2g6c
3,2500021,Churchill War Rooms,London,landmarks,https://www.youtube.com/watch?v=bEC-a8CKHJ8
4,2700023,Thames Rockets,London,outdoor_activities,https://www.youtube.com/watch?v=n3xy6sp3iqg
5,2700011,Fat Tire Tours Berlin,Berlin,outdoor_activities,https://www.youtube.com/watch?v=qYKkulHdfHY
6,2800006,Musee d'Orsay,Paris,museum,https://www.youtube.com/watch?v=nwSMOrwH9IA
7,2800021,National Gallery,London,museum,https://www.youtube.com/watch?v=TsSKnRIk0S8


In [ ]:
out = []
for c in category:
    t = links[links.category == c]
    for row in t.to_dict('records'):
        filename = str(row['product_id'])+'_'+row['URLS'].split('=')[-1]+'.en.srt'
        file_clean = filename.split('.srt')[0]+'_cleaned.srt'
        if not os.path.exists(file_clean):
            print "not"
            text = ""
            command ="youtube-dl -o "+str(row['product_id'])+'_'+row['URLS'].split('=')[-1]+".mp4 --sub-format vtt --convert-subs srt --write-auto-sub --sub-lang en -f mp4 "+str(row['URLS']).strip()
            subprocess.call(command, shell=True)
            subs = pysrt.open(filename, encoding='iso-8859-1')
            f =open(file_clean,'a+')
            j=0
            row['start_time'] = subs[0].start
            for i in range(0,len(subs)):
                if i%2 !=0:
                    text = text+subs[i].text+" "
                    print >> f,j
                    print >> f,subs[i-1].start,' --> ',subs[i-1].end
                    print >> f,subs[i].text,'\n'
                    j =j +1
            os.remove(filename)
            f.close()
        else:
            print "yes"
            text =""
            subs = pysrt.open(file_clean, encoding='iso-8859-1')
            for i in range(0,len(subs)):
                text = text+subs[i].text+" "

            row['start_time'] = subs[0].start 
        row['end_time'] = subs[len(subs)-1].end
        row['review_text'] = text 
        row['source'] = 7
        row['source_product_id'] = row['product_id']
        row['source_review_id'] = row['URLS'].split('=')[-1]
        out.append(row)
    video_review = pd.DataFrame(out)
    video_review=video_review.rename(index=str, columns={"source_product_name": "product_name","URLS": "review_url"})
    video_review['review_date'] = ""
    video_review['star_rating'] = 5
    video_review['verified_user']=""
    video_review['reviewer_name']=""
    video_review['review_url']=video_review['review_url']
    video_review['review_tag']=""
    col = ['source','source_review_id','source_product_id','product_name','review_date','star_rating','verified_user','reviewer_name','review_url','review_tag','review_text']
    video_review.to_csv(c+'_source_video_reviews.txt',sep='~',quoting=csv.QUOTE_NONE,encoding='utf-8',index=False,columns=col)

In [ ]:
video_snip

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from itertools import *
file =sys.argv[1]
video_snip = pd.read_csv(file,sep='~')
c = file.split('_')[3]
ids= video_snip.source_product_id.unique()
timing_output=[]
flag=0
t1=""
t2=""
t3=""
t4=""
t5=""
for id in ids:
    temp = video_snip[video_snip.source_product_id == id]
    for row in temp.to_dict('records'):
        x=""
        row['video_start_time'] = str(subs[0].start).split(',')[0]
        subs = pysrt.open(c+'_'+str(id)+'_'+row['source_review_id'].split('=')[-1]+'.en_cleaned.srt', encoding='iso-8859-1')
        for i in range(0,len(subs)):
            subs[i].text = subs[i].text.lower().replace('-',' ')
            subs[i].text = re.sub(r'([^\s\w]|_)+', '',subs[i].text )
            row['sentiment_text'] = row['sentiment_text'].lower()
            row['sentiment_text'] = re.sub(r'([^\s\w]|_)+', '',row['sentiment_text'] )
            if row['sentiment_text'] in subs[i].text:
                row['start_time'] = str(subs[i-1].start).split(',')[0]
                row['end_time'] = str(subs[i].end).split(',')[0]
                x = subs[i].text
            else:
               # row['snippet_end_time'] = subs[i].end 
                if i-1 >=0:
                    t1 = subs[i-1].text+" "+subs[i].text
                if i-2 >=0:
                    t4 = subs[i-2].text+" "+subs[i-1].text+" "+subs[i].text
                if (i+1) < len(subs):
                    t2 = subs[i].text+" "+subs[i+1].text
                if (i+2) < len(subs):
                    t3 = subs[i].text+" "+subs[i+1].text+" "+subs[i+2].text
                if (i+3) < len(subs):
                    t5 = subs[i].text+" "+subs[i+1].text+" "+subs[i+2].text+" "+subs[i+3].text

                if row['sentiment_text'] in t1:
                    row['start_time'] = str(subs[i-1].start).split(',')[0]
                    row['end_time'] = str(subs[i].end).split(',')[0]
                    break
                elif row['sentiment_text'] in t2:
                    row['start_time'] = str(subs[i].start).split(',')[0]
                    row['end_time'] = str(subs[i+1].end).split(',')[0]
                    break
                elif row['sentiment_text'] in t3:
                    if (i+2) < len(subs):
                        row['start_time'] = str(subs[i].start).split(',')[0]
                        row['end_time'] = str(subs[i+2].end).split(',')[0]
                        break
                elif row['sentiment_text'] in t4:
                    row['start_time'] = str(subs[i-2].start).split(',')[0]
                    row['end_time'] = str(subs[i].end).split(',')[0]
                    break
                elif row['sentiment_text'] in t5:
                    row['start_time'] = str(subs[i].start).split(',')[0]
                    row['end_time'] = str(subs[i+3].end).split(',')[0]
                    break
        row['video_end_time'] = str(subs[len(subs)-1].end).split(',')[0]
        timing_output.append(row)
        row["sentiment_text_start_index"]=row['start_index']
        row["sentiment_text_end_index"]=row['end_index']
        row['snippet_length'] = (len(row["sentiment_text"]))
        
timing_output = pd.DataFrame(timing_output)                
timing_output['source_id'] = 7
timing_output=timing_output.rename(index=str, columns={"category": "category_id", "source_product_id": "product_id",'confidence-score':"sentiment_conf_score"})
COLS= ['product_id','category_id','source_id','source_review_id','video_review_id','category_aspect_id','aspect','sentiment_type','sentiment_text','sentiment_text_start_index','sentiment_text_end_index','sentiment_start_snippet','sentiment_end_snippet','treemap_name','sentiment_conf_score','end_time','start_time']
timing_output.to_csv(c+'_product_video_review_sentiment.csv',index=False,columns=COLS)
snipp = timing_output[['source_review_id','start_time','end_time','video_end_time','video_start_time','sentiment_text','snippet_length','source_id','product_id']]
snipp = snipp.rename(index=str, columns={"source_review_id": "video_loc", "sentiment_text": "snippet"})
col = ['id','video_loc','start_time','end_time','snippet','snippet_length']
snipp.to_csv(c+'_product_video_review_snippet.csv',index=False,columns=col)
colum =['id','product_id','video_loc','source_id','overall_rating','outof_rating','review_date','normalized_date','source_review_id','reviewer','review_title','review_url','video_url','review_text','video_review_text','start_time','end_time','priority','status']
vid_rev = snipp[['video_loc','video_end_time','video_start_time','source_id','product_id']]
vid_rev = vid_rev.rename(index=str, columns={"video_end_time": "end_time", "video_start_time": "start_time"})
vid_rev = vid_rev.drop_duplicates()
vid_rev['status']= 'A'
vid_rev.to_csv(c+'_product_video_review.csv',index=False,columns=colum)    

In [ ]:
snipp

In [ ]:
timing_output

In [ ]:
timing_output=timing_output.rename(index=str, columns={"category": "category_id", "source_product_id": "product_id",'confidence-score':"sentiment_conf_score"})
COLS= ['product_id','category_id','source_id','source_review_id','video_review_id','category_aspect_id','aspect','sentiment_type','sentiment_text','sentiment_text_start_index','sentiment_text_end_index','sentiment_start_snippet','sentiment_end_snippet','treemap_name','sentiment_conf_score','end_time','start_time']
timing_output.to_csv('product_video_review_sentiment.csv',index=False,columns=COLS)

In [ ]:
snipp = timing_output[['source_review_id','start_time','end_time','video_end_time','video_start_time','sentiment_text','snippet_length','source_id','product_id']]
snipp = snipp.rename(index=str, columns={"source_review_id": "video_loc", "sentiment_text": "snippet"})
col = ['id','video_loc','start_time','end_time','snippet','snippet_length']

In [ ]:
timing_output

In [ ]:
snipp.to_csv('product_video_review_snippet.csv',index=False,columns=col)

In [ ]:
colum =['id','product_id','video_loc','source_id','overall_rating','outof_rating','review_date','normalized_date','source_review_id','reviewer','review_title','review_url','video_url','review_text','video_review_text','start_time','end_time','priority','status']
vid_rev = snipp[['video_loc','video_end_time','video_start_time','source_id','product_id']]
vid_rev = vid_rev.rename(index=str, columns={"video_end_time": "end_time", "video_start_time": "start_time"})

In [ ]:
vid_rev = vid_rev.drop_duplicates()
vid_rev['status']= 'A'
vid_rev.to_csv('product_video_review.csv',index=False,columns=colum)

In [ ]:
vid_rev

In [ ]:
x = pd.read_csv('lanmarks_video_reviews.csv')

In [ ]:
import csv
x.to_csv('landmarks_video_reviews.txt',sep='~',quoting=csv.QUOTE_NONE,encoding='utf-8',index=False)